최종 완성본

In [22]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace generative.ipynb

[NbConvertApp] WARNING | pattern 'generative.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

file_path = '/content/drive/My Drive/기말 텍스트마이닝/cleaned_combined_df.pkl'

try:
    with open(file_path, 'rb') as f:
        combined_df = pickle.load(f)

    # 5. 불러온 데이터 확인 (선택 사항)
    print("파일을 성공적으로 불러왔습니다.")
    # print(data) # 데이터 내용을 출력하려면 이 줄의 주석을 해제하세요.

except FileNotFoundError:
    print(f"오류: '{file_path}' 경로에 파일이 없습니다. 경로를 다시 확인해주세요.")

<ipython-input-3-2642707429>:7: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  combined_df = pickle.load(f)


파일을 성공적으로 불러왔습니다.


In [ ]:
# 키워드 입력 받기"
keywords = input('가장 관심있는 키워드를 첫 번째에 입력하고, 첫 번째 키워드와 관련된 보조 키워드를 뒤에 입력하세요. 키워드는 공백으로 구분하여 입력하세요:').split()

가장 관심있는 키워드를 첫 번째에 입력하고, 첫 번째 키워드와 관련된 보조 키워드를 뒤에 입력하세요. 키워드는 공백으로 구분하여 입력하세요:김건희 명태균


In [ ]:
# 모든 키워드를 포함하는 기사인지 확인하는 함수
def contains_all_keywords(row, keywords):
    text = row['title'] + ' ' + row['content']
    return all(kw in text for kw in keywords)

# 키워드 포함하는 데이터프레임을 filtered_data로 저장
filtered_data = combined_df[
    combined_df.apply(lambda row: contains_all_keywords(row, keywords), axis=1)
].reset_index(drop=True)

from datetime import datetime, timedelta

# 1. 오전/오후 → AM/PM으로 바꾸고 공백 제거
filtered_data['created_date_parsed'] = filtered_data['created_date'].str.replace('오전', 'AM', regex=False)
filtered_data['created_date_parsed'] = filtered_data['created_date_parsed'].str.replace('오후', 'PM', regex=False)

# 2. 날짜 형식 변환 (예: "2025.05.23. PM 1:00" → datetime)
filtered_data['created_date_parsed'] = pd.to_datetime(
    filtered_data['created_date_parsed'], format="%Y.%m.%d. %p %I:%M", errors='coerce'
)

# 3. created_date 기준으로 내림차순 정렬
filtered_data = filtered_data.sort_values(by='created_date', ascending=False).reset_index(drop=True)

# 4. 만약 필터링된 기사가 500개를 초과하면 → 1달 이내 기사만 추출
if len(filtered_data) > 500:
    # 최신 기사 날짜 기준 1달 전 계산
    latest_date = pd.to_datetime(filtered_data['created_date_parsed'].iloc[0])
    one_month_ago = latest_date - timedelta(days=30)

    # 1달 이내 기사만 추출
    filtered_data = filtered_data[
        pd.to_datetime(filtered_data['created_date_parsed']) >= one_month_ago
    ].reset_index(drop=True)

# 5. 결과 확인
print(f"최종 선택된 기사 수: {len(filtered_data)}")
print(filtered_data[['press', 'created_date', 'title']].head())

최종 선택된 기사 수: 65
  press         created_date  \
0  한국일보  2025.05.23. 오후 1:00   
1  중앙일보  2025.05.23. 오전 9:30   
2  한국일보  2025.05.23. 오전 9:00   
3  한국일보  2025.05.23. 오전 4:31   
4  한국일보  2025.05.23. 오전 4:31   

                                               title  
0  보수도 "명태균·김건희특검 둘 다 추진해야"... '김문수 강경 지지층만' 반대[한...  
1               "김건희 본격 수사는 대선 이후…잔인한 6月 될 것" [월간중앙]  
2  차기 대통령 임기는 '5년' 선호... 개헌은 내년 34%, 3년 뒤 28%[한국일...  
3   71%가 지지한 검찰개혁은 '시기의 문제'... 공수처 존폐는 팽팽[한국일보 여론조사]  
4  '선택적 모병제' 71% 찬성, '기본소득'은 39% 그쳐... 국민이 원하는 대선...  


In [ ]:
!pip install -q transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00


In [ ]:
# 1. 임포트
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib.pyplot as plt
import pandas as pd

# 2. SBERT 임베딩
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
texts = filtered_data['text'].tolist()
sbert_embeddings = model.encode(texts, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 3. 최적의 K 찾기 (silhouette)
sil_scores = []
K_range = range(2, min(11, len(texts)))  # 데이터 수가 적을 경우 대비
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(sbert_embeddings)
    score = silhouette_score(sbert_embeddings, labels)
    sil_scores.append(score)

# 최적 K 선택
best_k = K_range[sil_scores.index(max(sil_scores))]
print(f"✅ 최적 K 값: {best_k}")

✅ 최적 K 값: 4


In [ ]:
# 4. 최종 KMeans 수행 및 중심 기사 선택
kmeans = KMeans(n_clusters=best_k, random_state=42)
cluster_labels = kmeans.fit_predict(sbert_embeddings)
cluster_centers = kmeans.cluster_centers_

# 군집 중심과 가장 가까운 기사 인덱스
selected_indices, _ = pairwise_distances_argmin_min(cluster_centers, sbert_embeddings)
print(f"✅ 군집별 대표 기사 인덱스: {selected_indices}")

✅ 군집별 대표 기사 인덱스: [59 43  0 55]


In [ ]:
######추출된 기사 확인용 코드 (미시행해도 최종 결과에 지장 X)
# 대표 기사 정보 추출 (요약 없이 기사 본문 포함)
summary_rows = []
for idx in selected_indices:
    row = filtered_data.iloc[idx]
    summary_rows.append({
        "언론사": row['press'],
        "작성일자": row['created_date'],
        "제목": row['title'],
        "기사 본문": row['content'],
        "기사 전문 보기": row['url']
    })

# 데이터프레임 변환
summary_df = pd.DataFrame(summary_rows)

# 엑셀 파일로 저장
output_file = "대표_기사_목록.xlsx"
summary_df.to_excel(output_file, index=False)

print(f"✅ 엑셀 파일 저장 완료: {output_file}")

✅ 엑셀 파일 저장 완료: 대표_기사_목록.xlsx


In [ ]:
######추출된 기사 확인용 코드 (미시행해도 최종 결과에 지장 X)
import os
import shutil

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 2. 저장 경로 설정
drive_dir = '/content/drive/MyDrive/Colab 결과'
os.makedirs(drive_dir, exist_ok=True)

# 3. 파일 복사
shutil.copy("대표_기사_목록.xlsx", os.path.join(drive_dir, "대표_기사_목록.xlsx"))

print(f"✅ Google Drive 저장 완료: {drive_dir}/대표_기사_목록.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 저장 완료: /content/drive/MyDrive/Colab 결과/대표_기사_목록.xlsx


In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm

# tqdm 연동
tqdm.pandas()

# 1. 모델 및 토크나이저 로딩
model_name = "digit82/kobart-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 2. 요약 함수 정의
def summarize_kobart(text):
    # 예외 처리: 빈 문장이나 짧은 문장은 요약 생략
    if not isinstance(text, str) or len(text.strip()) < 10:
        return ""

    input_ids = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(
        input_ids,
        max_length=100,
        min_length=20,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 3. 요약 수행
filtered_data['kobart_summary'] = filtered_data['text'].progress_apply(summarize_kobart)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
100%|██████████| 65/65 [10:37<00:00,  9.80s/it]


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
# ✅ 대표 기사 인덱스 사용
representative_articles = filtered_data.loc[selected_indices].copy()

# ✅ 필요한 열만 선택 및 이름 변경
output_df = representative_articles[['press', 'created_date', 'title', 'kobart_summary', 'url']].copy()
output_df.columns = ['언론사', '게재일자', '제목', '기사 요약', '기사 전문 보기']

# ✅ 최상단 문구
headline = f'"{keywords}"을 포함하는 기사들 중 가장 다른 논조의 기사들입니다.'

# ✅ Google Drive 저장 경로
excel_path = "/content/drive/My Drive/Colab 결과/생성형 요약.xlsx"

# ✅ 엑셀 파일로 저장
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    output_df.to_excel(writer, sheet_name='요약결과', startrow=1, index=False)

    workbook  = writer.book
    worksheet = writer.sheets['요약결과']
    worksheet.write(0, 0, headline)

print(f"✅ 엑셀 파일이 성공적으로 저장되었습니다: {excel_path}")

✅ 엑셀 파일이 성공적으로 저장되었습니다: /content/drive/My Drive/Colab 결과/생성형 요약.xlsx
